### 1. Bibliotecas do Projeto

In [1]:
import os
import zipfile
import dask.dataframe as dd
import shutil
from glob import glob


### 2. Funções do Projeto para Importação dos Dados

In [2]:
# Função para processar e salvar CSV como Parquet usando Dask
def process_csv_with_dask(zip_file, zip_info, parquet_file, column_names):
    """
    Lê um arquivo CSV extraído de um arquivo ZIP usando Dask e salva diretamente em um arquivo Parquet.
    Recebe os nomes das colunas como parâmetros.
    """
    try:
        # Extrair o arquivo CSV para um local temporário
        temp_csv_path = f"./temp_{zip_info.filename}"
        
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            with open(temp_csv_path, 'wb') as temp_file:
                temp_file.write(zip_ref.read(zip_info.filename))

        # Forçar todas as colunas a serem lidas como strings
        df = dd.read_csv(temp_csv_path, sep=';', encoding='ISO-8859-1', dtype=str, header=None, on_bad_lines='skip')

        # Definir os nomes das colunas dinamicamente
        df.columns = column_names

        # Converter diretamente para Parquet usando Dask
        df.to_parquet(parquet_file, write_index=False)

        print(f"Dados gravados em {parquet_file} com sucesso.")
        
        # Remover o arquivo temporário após o processamento
        os.remove(temp_csv_path)
    
    except Exception as e:
        print(f"Erro ao processar {zip_info.filename}: {e}")

# Função para processar todos os arquivos .zip e gravar os CSVs em Parquet
def process_all_files_with_dask(download_dir, output_dir, file_pattern, column_names, parquet_name):
    """
    Processa todos os arquivos .zip encontrados no diretório de download, descompacta e converte os CSVs para Parquet.
    Se o arquivo Parquet correspondente já existir, ele será deletado e recriado.
    """
    # Define o caminho completo do arquivo Parquet de saída
    parquet_file = os.path.join(output_dir, parquet_name + ".parquet")

    # Verifica se o arquivo Parquet já existe e deleta se necessário
    if os.path.exists(parquet_file):
        print(f"O arquivo {parquet_file} já existe. Deletando...")

        # Se o arquivo for um diretório, use rmtree, senão remova o arquivo
        if os.path.isdir(parquet_file):
            shutil.rmtree(parquet_file)  # Deleta o diretório Parquet existente
        else:
            os.remove(parquet_file)  # Deleta o arquivo Parquet existente

    # Usar glob para encontrar todos os arquivos zip que correspondem ao padrão
    zip_files = glob(os.path.join(download_dir, '*.zip'))

    for zip_file in zip_files:
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                for zip_info in zip_ref.infolist():
                    # Verifica se o arquivo contém o padrão informado (file_pattern)
                    if file_pattern in zip_info.filename:
                        print(f"Extraindo e convertendo {zip_info.filename} de {zip_file} para Parquet...")

                        # Processa o CSV com Dask usando os nomes de colunas dinamicamente
                        process_csv_with_dask(zip_file, zip_info, parquet_file, column_names)

        except zipfile.BadZipFile as e:
            print(f"Erro: {zip_file} não é um arquivo .zip válido. Ignorando...")
        except Exception as e:
            print(f"Erro ao processar {zip_file}: {e}")


### 3. Lista das Colunas das Tabelas

In [3]:
# Definir os nomes das colunas para o arquivo EMPRESAS
cols_empresas = [
    'cnpj_basico',
    'razao_social',
    'natureza_juridica',
    'qualificacao_responsavel',
    'capital_social',
    'porte',
    'ente_federativo_responsavel'
]

# Colunas da Tabela "Estabelecimentos"
cols_estabelecimentos = [
    'cnpj_basico',
    'cnpj_ordem',
    'cnpj_dv',
    'identificador_matriz_filial',
    'nome_fantasia',
    'situacao_cadastral',
    'data_situacao_cadastral',
    'motivo_situacao_cadastral',
    'nome_cidade_exterior',
    'pais',
    'data_inicio_atividade',
    'cnae_fiscal_principal',
    'cnae_fiscal_secundaria',
    'tipo_logradouro',
    'logradouro',
    'numero',
    'complemento',
    'bairro',
    'cep',
    'uf',
    'municipio',
    'ddd1',
    'telefone1',
    'ddd2',
    'telefone2',
    'ddd_fax',
    'fax',
    'email',
    'situacao_especial',
    'data_situacao_especial'
]

# Colunas da Tabela "Socios"
cols_socios = [
    'cnpj_basico',
    'identificador_socio',
    'nome_socio',
    'cnpj_cpf_socio',
    'qualificacao_socio',
    'data_entrada_sociedade',
    'pais',
    'cpf_representante_legal',
    'nome_representante',
    'qualificacao_representante',
    'faixa_etaria'
]


# Colunas da Tabela "CNAE"
cols_cnae = [
    'codigo',
    'descricao'
]

# Colunas da Tabela "Motivos"
cols_motivos = [
    'codigo',
    'descricao'
]

# Colunas da Tabela "Municipios"
cols_municipios = [
    'codigo',
    'descricao'
]

# Colunas da Tabela "Natureza Jurídica"
cols_natureza_juridica = [
    'codigo',
    'descricao'
]

# Colunas da Tabela "Países"
cols_paises = [
    'codigo',
    'descricao'
]

# Colunas da Tabela "Qualificações"
cols_qualificacoes = [
    'codigo',
    'descricao'
]

# Colunas da Tabela "Simples Nacional"
cols_simples = [
    'cnpj_basico',
    'opcao_simples',
    'data_opcao_simples',
    'data_exclusao_simples',
    'opcao_mei',
    'data_opcao_mei',
    'data_exclusao_mei'
]


### 4. Processamento das Arquivos CSV to Parquet

In [4]:
# Processar todos os arquivos de EMPRESAS
process_all_files_with_dask(
    download_dir='./downloads/',    # Diretório onde os arquivos zip estão
    output_dir='./parquets/',       # Diretório de saída para os Parquet
    file_pattern='EMPRECSV',        # Padrão para encontrar os arquivos específicos
    column_names=cols_empresas,  # Nomes das colunas
    parquet_name='empresas'              # Nome do arquivo Parquet de saída
)


O arquivo ./parquets/empresas.parquet já existe. Deletando...
Extraindo e convertendo K3241.K03200Y2.D40914.EMPRECSV de ./downloads/Empresas2.zip para Parquet...
Dados gravados em ./parquets/empresas.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y3.D40914.EMPRECSV de ./downloads/Empresas3.zip para Parquet...
Dados gravados em ./parquets/empresas.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y1.D40914.EMPRECSV de ./downloads/Empresas1.zip para Parquet...
Dados gravados em ./parquets/empresas.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y0.D40914.EMPRECSV de ./downloads/Empresas0.zip para Parquet...
Dados gravados em ./parquets/empresas.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y4.D40914.EMPRECSV de ./downloads/Empresas4.zip para Parquet...
Dados gravados em ./parquets/empresas.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y5.D40914.EMPRECSV de ./downloads/Empresas5.zip para Parquet...
Dados gravados em ./parquets/empresas.parqu

In [5]:
# Processar todos os arquivos de SOCIOS
process_all_files_with_dask(
    download_dir='./downloads/',    # Diretório onde os arquivos zip estão
    output_dir='./parquets/',       # Diretório de saída para os Parquet
    file_pattern='SOCIOCSV',        # Padrão para encontrar os arquivos específicos
    column_names=cols_socios,       # Nomes das colunas
    parquet_name='socios'           # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/socios.parquet já existe. Deletando...
Extraindo e convertendo K3241.K03200Y4.D40914.SOCIOCSV de ./downloads/Socios4.zip para Parquet...
Dados gravados em ./parquets/socios.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y5.D40914.SOCIOCSV de ./downloads/Socios5.zip para Parquet...
Dados gravados em ./parquets/socios.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y7.D40914.SOCIOCSV de ./downloads/Socios7.zip para Parquet...
Dados gravados em ./parquets/socios.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y6.D40914.SOCIOCSV de ./downloads/Socios6.zip para Parquet...
Dados gravados em ./parquets/socios.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y2.D40914.SOCIOCSV de ./downloads/Socios2.zip para Parquet...
Dados gravados em ./parquets/socios.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y3.D40914.SOCIOCSV de ./downloads/Socios3.zip para Parquet...
Dados gravados em ./parquets/socios.parquet com sucesso.
Extraindo 

In [6]:
# Processar todos os arquivos de ESTABELECIMENTOS
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='ESTABELE',                  # Padrão para encontrar os arquivos específicos
    column_names=cols_estabelecimentos,       # Nomes das colunas
    parquet_name='estabelecimentos'           # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/estabelecimentos.parquet já existe. Deletando...
Extraindo e convertendo K3241.K03200Y4.D40914.ESTABELE de ./downloads/Estabelecimentos4.zip para Parquet...
Dados gravados em ./parquets/estabelecimentos.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y5.D40914.ESTABELE de ./downloads/Estabelecimentos5.zip para Parquet...
Dados gravados em ./parquets/estabelecimentos.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y7.D40914.ESTABELE de ./downloads/Estabelecimentos7.zip para Parquet...
Dados gravados em ./parquets/estabelecimentos.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y6.D40914.ESTABELE de ./downloads/Estabelecimentos6.zip para Parquet...
Dados gravados em ./parquets/estabelecimentos.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y2.D40914.ESTABELE de ./downloads/Estabelecimentos2.zip para Parquet...
Dados gravados em ./parquets/estabelecimentos.parquet com sucesso.
Extraindo e convertendo K3241.K03200Y3.D40914.ESTABELE 

In [7]:
# Processar todos os arquivos de CNAE
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='CNAECSV',                   # Padrão para encontrar os arquivos específicos
    column_names=cols_cnae,                   # Nomes das colunas
    parquet_name='cnae'                       # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/cnae.parquet já existe. Deletando...
Extraindo e convertendo F.K03200$Z.D40914.CNAECSV de ./downloads/Cnaes.zip para Parquet...
Dados gravados em ./parquets/cnae.parquet com sucesso.


In [8]:
# Processar todos os arquivos de SIMPLES
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='SIMPLES',                   # Padrão para encontrar os arquivos específicos
    column_names=cols_simples,                # Nomes das colunas
    parquet_name='simples'                    # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/simples.parquet já existe. Deletando...
Extraindo e convertendo F.K03200$W.SIMPLES.CSV.D40914 de ./downloads/Simples.zip para Parquet...
Dados gravados em ./parquets/simples.parquet com sucesso.


In [9]:
# Processar todos os arquivos de MUNICIPIOS
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='MUNICCSV',                  # Padrão para encontrar os arquivos específicos
    column_names=cols_municipios,             # Nomes das colunas
    parquet_name='municipios'                 # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/municipios.parquet já existe. Deletando...
Extraindo e convertendo F.K03200$Z.D40914.MUNICCSV de ./downloads/Municipios.zip para Parquet...
Dados gravados em ./parquets/municipios.parquet com sucesso.


In [10]:
# Processar todos os arquivos de MOTIVOS
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='MOTICSV',                   # Padrão para encontrar os arquivos específicos
    column_names=cols_motivos,                # Nomes das colunas
    parquet_name='motivos'                    # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/motivos.parquet já existe. Deletando...
Extraindo e convertendo F.K03200$Z.D40914.MOTICSV de ./downloads/Motivos.zip para Parquet...
Dados gravados em ./parquets/motivos.parquet com sucesso.


In [11]:
# Processar todos os arquivos de NATUREZA JURIDICA
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='NATJUCSV',                  # Padrão para encontrar os arquivos específicos
    column_names=cols_natureza_juridica,      # Nomes das colunas
    parquet_name='natjuridicas'               # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/natjuridicas.parquet já existe. Deletando...
Extraindo e convertendo F.K03200$Z.D40914.NATJUCSV de ./downloads/Naturezas.zip para Parquet...
Dados gravados em ./parquets/natjuridicas.parquet com sucesso.


In [12]:

# Processar todos os arquivos de PAISES
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='PAISCSV',                   # Padrão para encontrar os arquivos específicos
    column_names=cols_paises,                 # Nomes das colunas
    parquet_name='paises'                     # Nome do arquivo Parquet de saída
)

O arquivo ./parquets/paises.parquet já existe. Deletando...
Extraindo e convertendo F.K03200$Z.D40914.PAISCSV de ./downloads/Paises.zip para Parquet...
Dados gravados em ./parquets/paises.parquet com sucesso.


In [13]:
# Processar todos os arquivos de QUALIFICACOES
process_all_files_with_dask(
    download_dir='./downloads/',              # Diretório onde os arquivos zip estão
    output_dir='./parquets/',                 # Diretório de saída para os Parquet
    file_pattern='QUALSCSV',                  # Padrão para encontrar os arquivos específicos
    column_names=cols_qualificacoes,          # Nomes das colunas
    parquet_name='qualificacoes'              # Nome do arquivo Parquet de saída
)


O arquivo ./parquets/qualificacoes.parquet já existe. Deletando...
Extraindo e convertendo F.K03200$Z.D40914.QUALSCSV de ./downloads/Qualificacoes.zip para Parquet...
Dados gravados em ./parquets/qualificacoes.parquet com sucesso.
